In [11]:
info = """
LangChain 1.0 - Validation & Retry (验证和重试)
===============================================

本模块重点讲解：
1. with_retry() - 自动重试机制
2. with_fallbacks() - 降级/备用方案
3. Pydantic 验证错误处理
4. 自定义验证逻辑
5. 重试循环实现

⚠️ 注意：
- with_structured_output() 可能在某些模型上不完全支持
- 如遇到错误，示例会显示错误信息并跳过
"""
print(info)


LangChain 1.0 - Validation & Retry (验证和重试)

本模块重点讲解：
1. with_retry() - 自动重试机制
2. with_fallbacks() - 降级/备用方案
3. Pydantic 验证错误处理
4. 自定义验证逻辑
5. 重试循环实现

⚠️ 注意：
- with_structured_output() 可能在某些模型上不完全支持
- 如遇到错误，示例会显示错误信息并跳过



In [31]:
import os 
import json
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from pydantic import BaseModel,Field,field_validator,ValidationError
from typing import Optional,List,TypeVar,Type
from enum import Enum
import time

In [32]:
# 加载环境变量
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY or GROQ_API_KEY == "your_groq_api_key_here":
    raise ValueError(
        "\n请先在 .env 文件中设置有效的 GROQ_API_KEY\n"
        "访问 https://console.groq.com/keys 获取免费密钥"
    )


In [35]:

# 初始化模型
model = init_chat_model("groq:llama-3.3-70b-versatile", api_key=GROQ_API_KEY)


In [36]:
model.invoke("你好")

AIMessage(content='你好，有什么需要我帮助你吗？', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 37, 'total_tokens': 48, 'completion_time': 0.034759186, 'completion_tokens_details': None, 'prompt_time': 0.004941407, 'prompt_tokens_details': None, 'queue_time': 0.069432205, 'total_time': 0.039700593}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_43d97c5965', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b44a0-93d2-7c03-b16c-a208d98f0d6a-0', usage_metadata={'input_tokens': 37, 'output_tokens': 11, 'total_tokens': 48})

In [37]:
T = TypeVar("T", bound=BaseModel)

In [38]:
def safe_structured_output(prompt: str, output_class: Type[T], llm=None) -> T:
    """
    安全的结构化输出函数
    """
    if llm is None:
        llm = model
    
    try:
        structured_llm = llm.with_structured_output(output_class)
        return structured_llm.invoke(prompt)
    except Exception as e:
        print(f"  ⚠️ with_structured_output 失败: {e}")
        print("  📝 使用 JSON 解析 fallback...")
    
    # Fallback: 手动 JSON 解析
    json_prompt = f"""{prompt}

请严格按照JSON格式返回，只返回JSON，不要其他文字。"""
    
    response = llm.invoke([HumanMessage(content=json_prompt)])
    content = response.content.strip()
    
    if "```json" in content:
        content = content.split("```json")[1].split("```")[0]
    elif "```" in content:
        content = content.split("```")[1].split("```")[0]
    
    try:
        data = json.loads(content.strip())
        return output_class.model_validate(data)
    except Exception as e2:
        raise ValueError(f"无法解析结构化输出: {e2}")




In [39]:
print("""
示例1：使用 with_retry() 处理网络错误

当遇到临时性错误（网络超时、API限流等）时自动重试
""")
print("\n" + "="*70)
print("示例 1：with_retry() - 自动重试机制")
print("="*70)



示例1：使用 with_retry() 处理网络错误

当遇到临时性错误（网络超时、API限流等）时自动重试


示例 1：with_retry() - 自动重试机制


In [40]:
# 创建带重试的LLM
llm_with_retry = model.with_retry(
    retry_if_exception_type=(ConnectionError, TimeoutError),  # 重试的异常类型
    wait_exponential_jitter=True,  # 指数退避加抖动
    stop_after_attempt=3,  # 最大重试次数
)

In [41]:
print("\n配置:")
print("  - 重试异常: ConnectionError, TimeoutError")
print("  - 最大重试次数: 3")
print("  - 退避策略: 指数退避 + 随机抖动")



配置:
  - 重试异常: ConnectionError, TimeoutError
  - 最大重试次数: 3
  - 退避策略: 指数退避 + 随机抖动


In [42]:
try:
    print("尝试使用带重试的LLM进行调用...")
    response = llm_with_retry.invoke("你好")
    print(f"  ✅ 响应: {response}")
    print("\n✅️ 调用成功\n")
except Exception as e:
    print(f"❌ 调用3次仍然失败: {e}\n")

尝试使用带重试的LLM进行调用...
  ✅ 响应: content='你好，我很高兴见到你。今天我能为你做什么？' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 37, 'total_tokens': 56, 'completion_time': 0.108996527, 'completion_tokens_details': None, 'prompt_time': 0.001007223, 'prompt_tokens_details': None, 'queue_time': 0.172178264, 'total_time': 0.11000375}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_45180df409', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019b44a0-af98-7400-b2f1-242e5fc07f51-0' usage_metadata={'input_tokens': 37, 'output_tokens': 19, 'total_tokens': 56}

✅️ 调用成功



In [44]:
print("""
示例2：使用 with_fallbacks() 实现降级

主模型失败时，自动切换到备用模型
""")
print("\n" + "="*70)
print("示例 2：with_fallbacks() - 降级/备用方案")
print("="*70)



示例2：使用 with_fallbacks() 实现降级

主模型失败时，自动切换到备用模型


示例 2：with_fallbacks() - 降级/备用方案


In [57]:
primary_model = model
# 备用模型（更可靠或更便宜）
fallback_model = init_chat_model("groq:llama-3.1-8b-instant")


In [58]:
llm_with_fallbacks = primary_model.with_fallbacks([fallback_model])

In [59]:

print("\n配置:")
print("  - 主模型: llama-3.3-70b-versatile")
print("  - 备用模型: llama-3.1-8b-instant")


配置:
  - 主模型: llama-3.3-70b-versatile
  - 备用模型: llama-3.1-8b-instant


In [60]:
try:
    response = llm_with_fallbacks.invoke("用一句话介绍 Python")
    print(f"\n响应: {response.content}")
    print("\n关键点:")
    print("  - 主模型成功 → 使用主模型响应")
    print("  - 主模型失败 → 自动切换到备用模型")
    print("  - 适用于高可用性场景")
except Exception as e:
    print(f"\n所有模型都失败: {e}")


响应: Python是一种高级、解释型编程语言，以其简单、易读和多功能而闻名，使其成为初学者和经验丰富的开发人员在网页开发、数据分析、机器学习和自动化等广泛应用领域的热门选择。

关键点:
  - 主模型成功 → 使用主模型响应
  - 主模型失败 → 自动切换到备用模型
  - 适用于高可用性场景


In [61]:
class User(BaseModel):
    """用户信息（带验证）"""
    name: str = Field(description="姓名", min_length=2, max_length=20)
    age: int = Field(description="年龄", ge=0, le=150)  # 0-150 岁
    email: str = Field(description="邮箱")

    @field_validator('email')
    @classmethod
    def validate_email(cls, v):
        """自定义邮箱验证"""
        if '@' not in v:
            raise ValueError('邮箱必须包含 @')
        return v

In [62]:
print( """
    示例3：Pydantic 内置验证

    使用 Field 约束和自定义验证器
    """)


    示例3：Pydantic 内置验证

    使用 Field 约束和自定义验证器
    


In [63]:
print("\n" + "="*70)
print("示例 3：Pydantic 字段验证")
print("="*70)



示例 3：Pydantic 字段验证


In [64]:
print("\n" + "="*70)
print("示例 3：Pydantic 字段验证")
print("="*70)



示例 3：Pydantic 字段验证


In [65]:
print("\n测试 1: 有效数据")
try:
    user = User(name="张三", age=30, email="zhang@example.com")
    print(f"✓ 验证通过: {user.name}, {user.age}, {user.email}")
except ValidationError as e:
    print(f"✗ 验证失败: {e}")



测试 1: 有效数据
✓ 验证通过: 张三, 30, zhang@example.com


In [66]:
print("\n测试 2: 年龄超出范围")
try:
    user = User(name="李四", age=200, email="li@example.com")
    print(f"✓ 验证通过: {user}")
except ValidationError as e:
    print(f"✗ 验证失败: 年龄必须在 0-150 之间")
    print(f"   错误详情: {e.errors()[0]['msg']}")


测试 2: 年龄超出范围
✗ 验证失败: 年龄必须在 0-150 之间
   错误详情: Input should be less than or equal to 150


In [67]:
print("\n测试 3: 邮箱格式错误")
try:
    user = User(name="王五", age=25, email="invalid-email")
    print(f"✓ 验证通过: {user}")
except ValidationError as e:
    print(f"✗ 验证失败: 邮箱格式错误")
    print(f"   错误详情: {e.errors()[0]['msg']}")


测试 3: 邮箱格式错误
✗ 验证失败: 邮箱格式错误
   错误详情: Value error, 邮箱必须包含 @


In [68]:
print("\n关键点:")
print("  - Field(ge=, le=) - 数值范围约束")
print("  - Field(min_length=, max_length=) - 字符串长度约束")
print("  - @field_validator - 自定义验证逻辑")
print("  - ValidationError - 验证失败时抛出")


关键点:
  - Field(ge=, le=) - 数值范围约束
  - Field(min_length=, max_length=) - 字符串长度约束
  - @field_validator - 自定义验证逻辑
  - ValidationError - 验证失败时抛出


In [69]:
class Product(BaseModel):
    """产品信息（严格验证）"""
    name: str = Field(description="产品名称（字符串类型）", min_length=2)
    price: float = Field(description="价格，数字类型（必须 > 0）", gt=0)
    stock: int = Field(description="库存，整数类型（必须 >= 0）", ge=0)

    @field_validator('name')
    @classmethod
    def validate_name(cls, v):
        if v.lower() == "unknown":
            raise ValueError('产品名称不能是 unknown')
        return v

In [70]:


print("""
示例4：LLM 输出验证 + 重试循环

如果 LLM 输出不符合验证规则，重新提示并重试
""")
print("\n" + "="*70)
print("示例 4：LLM 输出验证 + 重试循环")
print("="*70)




示例4：LLM 输出验证 + 重试循环

如果 LLM 输出不符合验证规则，重新提示并重试


示例 4：LLM 输出验证 + 重试循环


In [71]:
structured_llm = model.with_structured_output(Product)

max_retries = 3

In [76]:
# 使用一个简单的测试案例（改为正常价格，避免触发验证错误）
text = "iPhone 15 售价 5999 元，库存 50 件"

print(f"\n提取文本: {text}")
print(f"验证规则: price > 0, stock >= 0, name 不能是 'unknown'\n")


提取文本: iPhone 15 售价 5999 元，库存 50 件
验证规则: price > 0, stock >= 0, name 不能是 'unknown'



In [77]:
for attempt in range(1, max_retries + 1):
    print(f"尝试 {attempt}/{max_retries}...")

    try:
        # 调用 LLM
        prompt = f"""从以下文本提取产品信息。
重要：price 必须是数字类型（不是字符串），stock 必须是整数类型。

文本: {text}"""
        result = structured_llm.invoke(prompt)

        # 如果到这里，说明验证通过
        print(f"✓ 提取成功!")
        print(f"  产品: {result.name}")
        print(f"  价格: {result.price} 元")
        print(f"  库存: {result.stock} 件")
        break

    except ValidationError as e:
        print(f"✗ Pydantic 验证失败: {e.errors()[0]['msg']}")

        if attempt < max_retries:
            error_msg = e.errors()[0]['msg']
            text = f"{text}\n注意: {error_msg}"
            print(f"  → 修正提示后重试...\n")
        else:
            print(f"  → 已达到最大重试次数")

    except Exception as e:
        # 捕获其他错误（如 BadRequestError）
        error_str = str(e)
        if "expected number, but got string" in error_str:
            print(f"✗ API 验证失败: LLM 返回了字符串而不是数字")
        elif "expected integer, but got string" in error_str:
            print(f"✗ API 验证失败: LLM 返回了字符串而不是整数")
        else:
            print(f"✗ 其他错误: {e}")

        if attempt < max_retries:
            print(f"  → 重试...\n")
            # 强化提示
            text = f"{text}\n重要: price 和 stock 必须是数字类型，不能是字符串"
        else:
            print(f"  → 已达到最大重试次数")
            print(f"\n说明: 某些模型可能会返回字符串类型的数字，")
            print(f"      这会导致 API 端验证失败。")


尝试 1/3...
✓ 提取成功!
  产品: iPhone 15
  价格: 5999.0 元
  库存: 50 件


In [78]:
print("\n关键点:")
print("  - ValidationError 捕获 Pydantic 验证失败")
print("  - Exception 捕获 API 端验证失败")
print("  - 在提示中强调类型要求")
print("  - 限制最大重试次数防止无限循环")


关键点:
  - ValidationError 捕获 Pydantic 验证失败
  - Exception 捕获 API 端验证失败
  - 在提示中强调类型要求
  - 限制最大重试次数防止无限循环


In [79]:
class Article(BaseModel):
    """文章信息"""
    title: str = Field(description="标题")
    content: str = Field(description="内容")
    word_count: int = Field(description="字数")

def validate_article(article: Article) -> bool:
    """
    自定义验证逻辑

    检查 word_count 是否与 content 实际字数接近
    """
    actual_count = len(article.content)
    claimed_count = article.word_count

    # 允许 10% 误差
    tolerance = 0.1
    lower_bound = actual_count * (1 - tolerance)
    upper_bound = actual_count * (1 + tolerance)

    if not (lower_bound <= claimed_count <= upper_bound):
        return False

    return True

In [80]:
print("""
示例5：自定义验证函数

Pydantic 验证之外的业务逻辑验证
""")
print("\n" + "="*70)
print("示例 5：自定义验证函数")
print("="*70)


示例5：自定义验证函数

Pydantic 验证之外的业务逻辑验证


示例 5：自定义验证函数


In [81]:
print("\n测试 1: 字数匹配")
article1 = Article(
    title="测试文章",
    content="这是一篇测试文章的内容",
    word_count=12
)

if validate_article(article1):
    print(f"✓ 验证通过: 声称字数 {article1.word_count}，实际 {len(article1.content)}")
else:
    print(f"✗ 验证失败: 字数不匹配")

print("\n测试 2: 字数不匹配（相差太大）")
article2 = Article(
    title="测试文章",
    content="短内容",
    word_count=1000  # 明显错误
)

if validate_article(article2):
    print(f"✓ 验证通过")
else:
    print(f"✗ 验证失败: 声称 {article2.word_count} 字，实际只有 {len(article2.content)} 字")



测试 1: 字数匹配
✓ 验证通过: 声称字数 12，实际 11

测试 2: 字数不匹配（相差太大）
✗ 验证失败: 声称 1000 字，实际只有 3 字


In [82]:
print("\n关键点:")
print("  - Pydantic 验证类型和格式")
print("  - 自定义函数验证业务逻辑")
print("  - 可以结合使用实现完整验证")



关键点:
  - Pydantic 验证类型和格式
  - 自定义函数验证业务逻辑
  - 可以结合使用实现完整验证


In [83]:
class ExtractedData(BaseModel):
    """提取的数据（完整验证）"""
    name: str = Field(description="名称（字符串类型）", min_length=1)
    value: float = Field(description="数值（数字类型，必须 > 0）", gt=0)

    @field_validator('name')
    @classmethod
    def validate_name(cls, v):
        if v.strip() == "":
            raise ValueError('名称不能为空')
        return v.strip()

In [85]:
def extract_with_validation(text: str, max_retries: int = 3) -> Optional[ExtractedData]:
    """
    带验证的提取函数

    Args:
        text: 待提取的文本
        max_retries: 最大重试次数

    Returns:
        提取的数据（验证通过）或 None（失败）
    """
    structured_llm = model.with_structured_output(ExtractedData)

    current_text = text

    for attempt in range(1, max_retries + 1):
        try:
            # 调用 LLM（强调类型）
            prompt = f"""提取以下文本中的信息。
重要：value 必须是数字类型（float），不能是字符串。

{current_text}"""
            result = structured_llm.invoke(prompt)

            # 额外的业务验证（Pydantic 已经检查了 gt=0）
            # 所有验证通过
            return result

        except ValidationError as e:
            error_msg = e.errors()[0]['msg']
            if attempt < max_retries:
                current_text = f"{text}\n\n注意：{error_msg}。请重新提取。"
            else:
                return None

        except Exception as e:
            # 捕获 API 错误
            if attempt < max_retries:
                current_text = f"{text}\n\n重要：value 必须是数字类型，不能是字符串。"
            else:
                return None

    return None

In [86]:
print("""
示例6：完整的验证 + 重试工作流

展示生产环境中的最佳实践
""")
print("\n" + "="*70)
print("示例 6：完整的验证 + 重试工作流")
print("="*70)



示例6：完整的验证 + 重试工作流

展示生产环境中的最佳实践


示例 6：完整的验证 + 重试工作流


In [87]:
test_cases = [
    "产品 A 的价值是 999.99 元",
    "产品 B 的价值是 1299 元",
]

for i, text in enumerate(test_cases, 1):
    print(f"\n--- 测试用例 {i} ---")
    print(f"文本: {text}")

    result = extract_with_validation(text, max_retries=2)

    if result:
        print(f"✓ 提取成功:")
        print(f"  名称: {result.name}")
        print(f"  数值: {result.value}")
    else:
        print(f"✗ 提取失败（重试 2 次后仍无法通过验证）")



--- 测试用例 1 ---
文本: 产品 A 的价值是 999.99 元
✓ 提取成功:
  名称: 产品 A
  数值: 999.99

--- 测试用例 2 ---
文本: 产品 B 的价值是 1299 元
✓ 提取成功:
  名称: 产品 B
  数值: 1299.0


In [88]:
print("\n关键点:")
print("  - 封装验证逻辑到函数中")
print("  - 清晰的错误处理")
print("  - 返回 Optional 表示可能失败")
print("  - 适合集成到生产系统")


关键点:
  - 封装验证逻辑到函数中
  - 清晰的错误处理
  - 返回 Optional 表示可能失败
  - 适合集成到生产系统


In [89]:
print("""
示例7：组合使用多种策略

网络重试 + 模型降级 + 输出验证

重要：调用顺序必须是：
1. 先 with_structured_output()
2. 再 with_retry()
3. 最后 with_fallbacks()
""")
print("\n" + "="*70)
print("示例 7：组合策略 - retry + fallbacks + validation")
print("="*70)


示例7：组合使用多种策略

网络重试 + 模型降级 + 输出验证

重要：调用顺序必须是：
1. 先 with_structured_output()
2. 再 with_retry()
3. 最后 with_fallbacks()


示例 7：组合策略 - retry + fallbacks + validation


In [90]:
# 1. 先创建结构化输出（必须先调用！）
structured_primary = model.with_structured_output(ExtractedData)

# 2. 配置备用模型（也要先创建结构化输出）
fallback_model = init_chat_model("groq:llama-3.3-70b-versatile", api_key=GROQ_API_KEY)
structured_fallback = fallback_model.with_structured_output(ExtractedData)

# 3. 添加重试（在结构化输出之后）
primary_with_retry = structured_primary.with_retry(
    retry_if_exception_type=(ConnectionError, TimeoutError),
    stop_after_attempt=2
)

# 4. 添加降级（最后一步）
robust_llm = primary_with_retry.with_fallbacks([structured_fallback])


In [91]:

print("\n配置（正确的调用顺序）:")
print("  1. 先创建结构化输出 (with_structured_output)")
print("  2. 再添加重试机制 (with_retry)")
print("  3. 最后添加降级方案 (with_fallbacks)")


配置（正确的调用顺序）:
  1. 先创建结构化输出 (with_structured_output)
  2. 再添加重试机制 (with_retry)
  3. 最后添加降级方案 (with_fallbacks)


In [92]:
try:
    prompt = """提取以下文本中的信息。
重要：value 必须是数字类型（float）。

产品 C 的价值是 1299 元"""
    result = robust_llm.invoke(prompt)
    print(f"\n✓ 成功提取:")
    print(f"  名称: {result.name}")
    print(f"  数值: {result.value}")
except Exception as e:
    print(f"\n✗ 所有策略都失败: {e}")



✓ 成功提取:
  名称: 产品 C
  数值: 1299.0


In [93]:
print("\n关键点:")
print("  - 调用顺序很重要！")
print("  - with_structured_output() 必须在最前面")
print("  - 然后是 retry、fallbacks")
print("  - 多层防护: 验证 → 重试 → 降级")
print("  - 生产环境推荐配置")


关键点:
  - 调用顺序很重要！
  - with_structured_output() 必须在最前面
  - 然后是 retry、fallbacks
  - 多层防护: 验证 → 重试 → 降级
  - 生产环境推荐配置
